## Load API Keys

In [32]:
from dotenv import load_dotenv
load_dotenv()

True

## Loading the document

In [37]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Minimalist CV Resume.pdf")
data = loader.load()

## Splitting/Chunking the document

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True,
)

texts = text_splitter.split_documents(data)

## Initialize Pinecone Client and the OpenAI embedding model

In [48]:
from langchain_openai.embeddings import OpenAIEmbeddings
from pinecone import Pinecone
import os

embeddings = OpenAIEmbeddings(openai_api_key= os.environ['OPENAI_API_KEY'])
pc = Pinecone(api_key = os.environ['PINECONE_API_KEY'])

# Check if index exist
index = pc.Index("ragchain-db")

index.delete(delete_all=True)

{}

## Insert embeddings to pinecone and set a retriever

In [40]:
from langchain.vectorstores import Pinecone as langpc
vectordb = langpc.from_documents(texts, embeddings, index_name = "ragchain-db")

In [41]:
retriever = vectordb.as_retriever()

## Initialize LLM

In [42]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo")
llm.invoke("What skills do Lorna Alvarado have?")

AIMessage(content="As an AI language model, I don't have access to personal information about individuals unless it has been shared with me during our conversation. Therefore, I don't have any information about a specific individual named Lorna Alvarado and her skills.")

## Initialize Memory and Chain

In [43]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm = llm, retriever= retriever, memory = memory)

## Inference LLM

In [47]:
query = "What is her references?"
chain.invoke({'question': query})

{'question': 'What is her references?',
 'chat_history': [HumanMessage(content='What skills do Lorna Alvarado have?'),
  AIMessage(content='The skills of Lorna Alvarado are not provided in the given information.'),
  HumanMessage(content='What work experience do Lorna Alvarado have?'),
  AIMessage(content='Lorna Alvarado has work experience as a Senior Graphic Designer at Fauguet Studio from 2013 to 2015, 2015 to 2017, and 2017 to 2019.'),
  HumanMessage(content='What is her contact details?'),
  AIMessage(content="Lorna Alvarado's contact details are as follows:\n\nPhone: +123-456-7890\nEmail: hello@reallygreatsite.com"),
  HumanMessage(content='What is her references?'),
  AIMessage(content='The references for Lorna Alvarado are not provided in the given context.')],
 'answer': 'The references for Lorna Alvarado are not provided in the given context.'}